In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('data/logit/bcancer.csv', index_col = 0, na_values='?')
data.tail()

,clump_thickness,uniformity_size,uniformity_shape,marginal_adhesion,epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
id,,,,,,,,,,
776715,3,1,1,1,3,2,1,1,1,2
841769,2,1,1,1,2,1,1,1,1,2
888820,5,10,10,3,7,3,8,10,2,4
897471,4,8,6,4,3,4,10,6,1,4
897471,4,8,8,5,4,5,10,4,1,4


In [3]:
#check for n/a
for attr in data.columns:
    print np.sum(data[attr].notnull())
#There are missing values in bare_nuclei column

#Drop na
data.dropna(axis=0, inplace=True)
print data.shape

699
699
699
699
699
683
699
699
699
699
(683, 10)


In [4]:
X = data[data.columns[:-1]]
y = data[data.columns[-1]]

In [5]:
#Normalize
X = (X - X.mean())/X.std()
y = y/2 - 1 #For whatever reason, results is stored as 4 for malignant, 2 for benign. Now 1 = malignant, 0 = benign

In [6]:
#Split into training and test data
mask = np.random.randn(data.shape[0]) < 0.8
X_train = X[mask]
y_train = y[mask]

X_test = X[~mask]
y_test = y[~mask]

In [7]:
def logistic_regression(X, y, iters=500):
    if len(X.shape) > 1:
        thetas = np.zeros(X.shape[1])
    else:
        thetas = np.zeros(1)
    alpha = 0.01
    Js =[]
    
    for i in xrange(iters):
        #Grab hypothesis
        if len(X.shape) > 1:
            h = g(X.dot(thetas))
        else:
            h = g(X*thetas)
        #Compute Js
        #J = -1 * np.sum(y*np.log(h) + (1-y)*np.log(1 - g(h))) / y.shape[0]
        #Js.append(J)
        #Update thetas
        thetas = thetas + alpha * (y - h).dot(X)
        
    _x = np.arange(iters)    
    #plt.plot(_x, Js)
    
    return thetas    
    
#Helper Functions
def g(Z):
    #Run logistic function on Z array
    return 1. / (1 + np.exp(-Z))

#Call
thetas = logistic_regression(X_train,y_train)

In [8]:
#Test
pred = g(X_test.dot(thetas)) >= 0.5
print np.sum(pred == y_test)/float(pred.shape[0])
print 'prediction rate seems pretty good'

0.961038961039
prediction rate seems pretty good


In [9]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train, y_train)
_pred = clf.predict(X_test)
compare = pd.DataFrame({'prediction': _pred, 'true': y_test})
print clf.score(X_test, y_test)

0.948051948052


In [10]:
df = pd.DataFrame({})
#Get predictors by attr
for attr in data.columns[:-1]:
    _t = logistic_regression(X_train[attr], y_train)
    df[attr] = g(X_test[attr] * _t) >= 0.5

df['logistic_regression'] = pred
    
print df.head()

         clump_thickness  uniformity_size  uniformity_shape  \
id                                                            
1002945             True             True              True   
1016277             True             True              True   
1017122             True             True              True   
1018561            False            False             False   
1036172            False            False             False   

         marginal_adhesion  epithelial_cell_size  bare_nuclei  \
id                                                              
1002945               True                  True         True   
1016277              False                 False         True   
1017122               True                  True         True   
1018561              False                 False        False   
1036172              False                 False        False   

         bland_chromatin  normal_nucleoli  mitoses logistic_regression  
id                           

In [11]:
for attr in data.columns[:-1]:
    df[attr + '_n'] = data[attr][~mask]

df['actual_y'] = y_test
df.columns = df.columns.map(lamb)
    
print df.head()

NameError: name 'lamb' is not defined